# __IOC Hunting__
### By: Alejandro Perez
----------------------------


## Introduction
This Notebook will be used as an test case where we will use the ThreatFox API
to pull the most recent IOC's that have been verified and then pivot from those
to using Data from other Sources such as VirusTotal, and GreyNoise



In [1]:
from src.ThreatFoxUtils import ThreatFox
from src.VirusTotalUtils import VirusTotal
from src.GreyNoiseUtils import Greynoise
import pandas as pd
from pandas import json_normalize
import ipywidgets as widgets
from IPython.display import clear_output
import json
import time

In [2]:
# This will use the Config File in order to 
with open('config.json') as config:
    configData = json.load(config)

In [3]:
# Instantiating the VirusTotal API Key from the config file and saving it in a Variable
VT = VirusTotal(ApiKey=configData['VirusTotal Api Key'])

In [4]:
# Gathering the IOC's for the past day
IocData = ThreatFox.query_ioc_database(number_of_days=1)

[-] Gathering IOC's from past 1 days...
[+] IOC Data has been Gathered


In [5]:
# Displaying 10 random Samples from the Dataset
IocData.sample(n=10)

,id,ioc,threat_type,threat_type_desc,ioc_type,ioc_type_desc,malware,malware_printable,malware_alias,malware_malpedia,confidence_level,first_seen,last_seen,reference,reporter,tags
69,7865,http://51.195.53.221/p.php/3bZQklG6hGLlX,botnet_cc,Indicator that identifies a botnet command&con...,url,URL that is used for botnet Command&control (C&C),win.lokipws,Loki Password Stealer (PWS),"Loki,LokiPWS,LokiBot",https://malpedia.caad.fkie.fraunhofer.de/detai...,100,2021-04-13 06:00:25 UTC,None,None,abuse_ch,[Loki]
5,7929,http://cupazo.co.in/TyBmo/index.php,botnet_cc,Indicator that identifies a botnet command&con...,url,URL that is used for botnet Command&control (C&C),win.azorult,Azorult,"PuffStealer,Rultazo",https://malpedia.caad.fkie.fraunhofer.de/detai...,100,2021-04-13 23:35:17 UTC,None,None,abuse_ch,[AZORult]
74,7861,http://94.140.115.43/1/,botnet_cc,Indicator that identifies a botnet command&con...,url,URL that is used for botnet Command&control (C&C),win.smokeloader,SmokeLoader,"Dofoil,Sharik,Smoke,Smoke Loader",https://malpedia.caad.fkie.fraunhofer.de/detai...,75,2021-04-13 05:50:08 UTC,None,https://bazaar.abuse.ch/sample/6b2e23e38be7ad2...,abuse_ch,[smokeloader]
73,7860,http://94.140.114.59/1/,botnet_cc,Indicator that identifies a botnet command&con...,url,URL that is used for botnet Command&control (C&C),win.smokeloader,SmokeLoader,"Dofoil,Sharik,Smoke,Smoke Loader",https://malpedia.caad.fkie.fraunhofer.de/detai...,75,2021-04-13 05:50:08 UTC,None,https://bazaar.abuse.ch/sample/06b686985f42468...,abuse_ch,[smokeloader]
62,7872,bf96b03f848b9893bcc13415f1cddb431a77c8c3551a79...,payload,Indicator that identifies a malware sample (pa...,sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/detai...,50,2021-04-13 07:03:29 UTC,None,https://twitter.com/RedBeardIOCs/status/138186...,Virus_Deck,[AgentTesla]
76,7858,45.32.99.249:7707,botnet_cc,Indicator that identifies a botnet command&con...,ip:port,ip:port combination that is used for botnet Co...,win.asyncrat,AsyncRAT,None,https://malpedia.caad.fkie.fraunhofer.de/detai...,75,2021-04-13 05:40:29 UTC,None,https://bazaar.abuse.ch/sample/1dcf43f57f670d1...,abuse_ch,[asyncrat]
17,7917,http://104.168.140.79/ghost/fre.php,botnet_cc,Indicator that identifies a botnet command&con...,url,URL that is used for botnet Command&control (C&C),win.lokipws,Loki Password Stealer (PWS),"Loki,LokiPWS,LokiBot",https://malpedia.caad.fkie.fraunhofer.de/detai...,100,2021-04-13 20:50:19 UTC,None,None,abuse_ch,[Loki]
38,7897,53fcaac5d5ca8caeb3d2e9b19a9e628b89fea8fb3f7dc7...,payload,Indicator that identifies a malware sample (pa...,sha256_hash,SHA256 hash of a malware sample (payload),win.nanocore,Nanocore RAT,"Nancrat,NanoCore",https://malpedia.caad.fkie.fraunhofer.de/detai...,50,2021-04-13 12:03:32 UTC,None,https://twitter.com/RedBeardIOCs/status/138194...,Virus_Deck,[NanoCore]
51,7887,6a3dbe59e320ddf283fdf1177f3345bba999e7b55e5c2f...,payload,Indicator that identifies a malware sample (pa...,sha256_hash,SHA256 hash of a malware sample (payload),win.formbook,Formbook,None,https://malpedia.caad.fkie.fraunhofer.de/detai...,50,2021-04-13 09:03:28 UTC,None,https://twitter.com/RedBeardIOCs/status/138189...,Virus_Deck,[Formbook]
47,7883,b5805e39212764c862be2a08240645a996b89526ab2fbb...,payload,Indicator that identifies a malware sample (pa...,sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/detai...,50,2021-04-13 09:03:28 UTC,None,https://twitter.com/RedBeardIOCs/status/138189...,Virus_Deck,[AgentTesla]


## __GreyNoise__
------------
In this Section, We will use Greynoise which a Service that will return information on IP Addresses in order to rule out any Whitenoise when it comes to scanning. Sources like Shodan, and Censys scan the Internet regularly but mean no harm when doing so, but this service will assist us in finding the IP Addresses that could be scanning maliciously, if there are any in the current dataset.

In order to do this we will take the Dataset from ThreatFox, Filter for IP Addresses, and then Apply it and have a Dataframe returned to us if there are any malicious IP Addresses to track.

In [6]:
# Filtering the Data Set for solely IP:Port Entries
Suspicious_IP = IocData[IocData['ioc_type']=='ip:port']

# Displaying the newly filtered Dataframe (only first 5 rows)
display(Suspicious_IP.head(5))

,id,ioc,threat_type,threat_type_desc,ioc_type,ioc_type_desc,malware,malware_printable,malware_alias,malware_malpedia,confidence_level,first_seen,last_seen,reference,reporter,tags
18,7916,92.62.115.177:80,botnet_cc,Indicator that identifies a botnet command&con...,ip:port,ip:port combination that is used for botnet Co...,win.fickerstealer,Ficker Stealer,None,https://malpedia.caad.fkie.fraunhofer.de/detai...,100,2021-04-13 20:16:48 UTC,None,None,abuse_ch,[FickerStealer]
19,7915,204.48.28.130:6606,botnet_cc,Indicator that identifies a botnet command&con...,ip:port,ip:port combination that is used for botnet Co...,win.asyncrat,AsyncRAT,None,https://malpedia.caad.fkie.fraunhofer.de/detai...,100,2021-04-13 16:05:13 UTC,None,None,abuse_ch,"[asyncrat, RAT]"
54,7880,176.111.174.14:20004,botnet_cc,Indicator that identifies a botnet command&con...,ip:port,ip:port combination that is used for botnet Co...,win.remcos,Remcos,"RemcosRAT,Remvio,Socmer",https://malpedia.caad.fkie.fraunhofer.de/detai...,75,2021-04-13 08:10:34 UTC,None,https://bazaar.abuse.ch/sample/39d5de9369e325d...,abuse_ch,[remcos]
63,7871,77.247.127.37:1777,botnet_cc,Indicator that identifies a botnet command&con...,ip:port,ip:port combination that is used for botnet Co...,win.bit_rat,BitRAT,None,https://malpedia.caad.fkie.fraunhofer.de/detai...,100,2021-04-13 06:45:27 UTC,None,None,abuse_ch,"[BitRAT, RAT]"
70,7864,185.140.53.9:9882,botnet_cc,Indicator that identifies a botnet command&con...,ip:port,ip:port combination that is used for botnet Co...,win.remcos,Remcos,"RemcosRAT,Remvio,Socmer",https://malpedia.caad.fkie.fraunhofer.de/detai...,100,2021-04-13 06:00:22 UTC,None,None,abuse_ch,"[RAT, RemcosRAT]"


In [7]:
# Creating List object to store GreyNoise data
GreyNoise_Data_List  = []

# Iterating through the Suspicious_IP Data
print("[-] Scanning IP's...")
for IP_Port in IocData['ioc']:
    # Splitting the IP:Port Combination for solely the IP Address
    IP = IP_Port.split(":")[0]
    # Using our Custom Function for GreyNoise to return the Information to us
    GreyNoise_Scan = Greynoise.lookup_IP(IP)
    # Checks to catch any errors such as this one and will pass over it
    if GreyNoise_Scan['message'] == "Request is not a valid routable IPv4 address":
        pass
    else:
        # Appending the Data to the list we made above
        GreyNoise_Data_List.append(GreyNoise_Scan)

print("[+] Scan has completed")


[-] Scanning IP's...
[+] Scan has completed


In [8]:
# Takes the List of Data and flattens the Json into a Dataframe
GreyNoiseDataframe = json_normalize(GreyNoise_Data_List)
# Displays the Dataframe
GreyNoiseDataframe

,ip,noise,riot,message,classification,name,link,last_seen
0,92.62.115.177,False,False,IP not observed scanning the internet or conta...,NaN,NaN,NaN,NaN
1,204.48.28.130,False,False,IP not observed scanning the internet or conta...,NaN,NaN,NaN,NaN
2,176.111.174.14,False,False,IP not observed scanning the internet or conta...,NaN,NaN,NaN,NaN
3,77.247.127.37,False,False,IP not observed scanning the internet or conta...,NaN,NaN,NaN,NaN
4,185.140.53.9,True,False,Success,malicious,unknown,https://viz.greynoise.io/ip/185.140.53.9,2021-02-12
5,45.32.99.249,False,False,IP not observed scanning the internet or conta...,NaN,NaN,NaN,NaN
6,45.32.99.249,False,False,IP not observed scanning the internet or conta...,NaN,NaN,NaN,NaN
7,45.32.99.249,False,False,IP not observed scanning the internet or conta...,NaN,NaN,NaN,NaN
8,45.32.99.249,False,False,IP not observed scanning the internet or conta...,NaN,NaN,NaN,NaN
9,213.152.187.210,True,False,Success,unknown,unknown,https://viz.greynoise.io/ip/213.152.187.210,2021-04-11


## __VirusTotal__
----------------
This Section is Dedicated to grabbing the Hashes from ThreatFox and pull reports from VirusTotal on them. In order to do this, we will follow the same methodology and filter out the hashes from our ThreatFox Dataset and then begin to pull information from VirusTotal.

However this section will be interactive with a small Widget tool to search for reports on the hash

In [9]:
# Filters for Suspicious Hashes
Suspicious_Hashes = IocData[IocData['ioc_type']=='sha256_hash']

In [10]:
# Displays the Suspicious Hashes first 5 rows
Suspicious_Hashes.head(5)

,id,ioc,threat_type,threat_type_desc,ioc_type,ioc_type_desc,malware,malware_printable,malware_alias,malware_malpedia,confidence_level,first_seen,last_seen,reference,reporter,tags
0,7934,a8f1fa1b80fda001331fc4cc87470e189c55653ac5d486...,payload,Indicator that identifies a malware sample (pa...,sha256_hash,SHA256 hash of a malware sample (payload),win.formbook,Formbook,None,https://malpedia.caad.fkie.fraunhofer.de/detai...,50,2021-04-14 02:06:13 UTC,None,https://twitter.com/RedBeardIOCs/status/138215...,Virus_Deck,[Formbook]
1,7933,82be061b2bfe48ee3a9f76ee99cf6f3ed712c0c1393ad4...,payload,Indicator that identifies a malware sample (pa...,sha256_hash,SHA256 hash of a malware sample (payload),win.formbook,Formbook,None,https://malpedia.caad.fkie.fraunhofer.de/detai...,50,2021-04-14 02:06:13 UTC,None,https://twitter.com/RedBeardIOCs/status/138215...,Virus_Deck,[Formbook]
2,7932,1f5992fbc69275585f817e70e7c8766e6b7a03e1b17386...,payload,Indicator that identifies a malware sample (pa...,sha256_hash,SHA256 hash of a malware sample (payload),win.formbook,Formbook,None,https://malpedia.caad.fkie.fraunhofer.de/detai...,50,2021-04-14 02:06:13 UTC,None,https://twitter.com/RedBeardIOCs/status/138215...,Virus_Deck,[Formbook]
3,7931,5af6eb871665b3bc59aea7a8bd187c80fecf392b7f5bc4...,payload,Indicator that identifies a malware sample (pa...,sha256_hash,SHA256 hash of a malware sample (payload),win.formbook,Formbook,None,https://malpedia.caad.fkie.fraunhofer.de/detai...,50,2021-04-14 02:06:13 UTC,None,https://twitter.com/RedBeardIOCs/status/138215...,Virus_Deck,[Formbook]
11,7923,f7d14c81b7dfd313809e9b0f49822ce77cf9b9913c6a45...,payload,Indicator that identifies a malware sample (pa...,sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/detai...,50,2021-04-13 22:06:16 UTC,None,https://twitter.com/RedBeardIOCs/status/138209...,Virus_Deck,[AgentTesla]


In [11]:
# Building the Output of the Widgets
VT_File_Report_Output = widgets.Output()

# Building the Hash Selection Dropdown Menu
VT_Hash_Selection = widgets.Dropdown(
    options = Suspicious_Hashes['ioc'].tolist(),
    description = 'Hashes:'
)

# Building the Button for the VT Hash Search
VT_SearchButton = widgets.Button(
    description = 'Search'
)

# adding action to the VT Search Button
def VT_Search_Action(x):
    with VT_File_Report_Output:
        clear_output()
        Report = VT.get_file_information(FileHash=VT_Hash_Selection.value)
        print(json.dumps(Report, indent=4, sort_keys=True))

# Attaching the Action Function to the button
VT_SearchButton.on_click(VT_Search_Action)

# Creating a Vertical Box to handle the widgets
widgets.VBox([VT_Hash_Selection, VT_SearchButton, VT_File_Report_Output])